# From SMILES to Production Conditions in 5 Lines of Python

Given any molecule as a SMILES string, MolBuilder can predict optimal reaction conditions --
selecting the best reaction template, analyzing steric and electronic properties, and computing
temperature, solvent, atmosphere, and yield estimates.

No RDKit. No C++ compilation. Just `pip install molbuilder`.

In [ ]:
# pip install molbuilder
from molbuilder.process.condition_prediction import predict_conditions

result = predict_conditions("CCO", reaction_name="oxidation", scale_kg=10.0)

print(f"Best reaction:  {result.best_match.template_name}")
print(f"Temperature:    {result.best_match.conditions.temperature_C} C")
print(f"Solvent:        {result.best_match.conditions.solvent}")
print(f"Atmosphere:     {result.best_match.conditions.atmosphere}")
print(f"Expected yield: {result.best_match.adjusted_yield_range[0]}-{result.best_match.adjusted_yield_range[1]}%")
print(f"Confidence:     {result.overall_confidence}")

## What just happened?

`predict_conditions()` ran a full analysis pipeline:

1. **Parsed** the SMILES string into a molecular graph
2. **Detected** functional groups (found: alcohol)
3. **Analyzed** steric environment (unhindered) and electronic character (neutral)
4. **Searched** 91 reaction templates for compatible oxidation reactions
5. **Scored** each candidate on FG match, yield, sterics, electronics, and compatibility
6. **Computed** optimized conditions for the top matches at 10 kg scale

Let's look at the substrate analysis:

In [ ]:
sa = result.substrate_analysis
print(f"Functional groups: {sa.detected_functional_groups}")
print(f"Molecular weight:  {sa.molecular_weight}")
print(f"Heavy atoms:       {sa.heavy_atom_count}")
print(f"Steric class:      {sa.steric_class}")
print(f"Electronic:        {sa.electronic_character}")
print(f"Sensitive groups:  {sa.sensitive_groups}")

## All ranked candidates

The engine doesn't just return one answer -- it ranks multiple viable reactions:

In [ ]:
for i, c in enumerate(result.candidates, 1):
    print(f"{i}. {c.template_name}")
    print(f"   Score: {c.match_score}/100  |  Category: {c.category}")
    print(f"   Temp: {c.conditions.temperature_C} C  |  Solvent: {c.conditions.solvent}")
    print(f"   Yield: {c.adjusted_yield_range[0]}-{c.adjusted_yield_range[1]}%")
    print()

## Solvent recommendations

Each candidate includes scored solvent alternatives with green chemistry ratings:

In [ ]:
for s in result.best_match.recommended_solvents:
    print(f"{s.solvent_name:25s}  score={s.composite_score:5.1f}  green={s.green_score}/10")

## Try a more complex molecule: ibuprofen

The system handles real drug molecules with multiple functional groups:

In [ ]:
ibu = predict_conditions(
    "CC(C)Cc1ccc(cc1)C(C)C(=O)O",  # ibuprofen
    reaction_name="oxidation",
    scale_kg=50.0,
)

print(f"Molecule:    ibuprofen (MW={ibu.substrate_analysis.molecular_weight})")
print(f"FGs found:   {ibu.substrate_analysis.detected_functional_groups}")
print(f"Steric:      {ibu.substrate_analysis.steric_class}")
print(f"Electronic:  {ibu.substrate_analysis.electronic_character}")
print(f"Sensitive:   {ibu.substrate_analysis.sensitive_groups}")
print()
print(f"Best match:  {ibu.best_match.template_name} (score={ibu.best_match.match_score})")
print(f"Conditions:  {ibu.best_match.conditions.temperature_C} C in {ibu.best_match.conditions.solvent}")
print(f"Yield:       {ibu.best_match.adjusted_yield_range[0]}-{ibu.best_match.adjusted_yield_range[1]}%")
print(f"Confidence:  {ibu.overall_confidence}")

## Without a reaction hint

You can also omit the `reaction_name` and let the engine figure out what's possible:

In [ ]:
# What reactions are possible on a primary alkyl halide?
r = predict_conditions("CCCl", scale_kg=1.0)

print(f"Substrate FGs: {r.substrate_analysis.detected_functional_groups}")
print(f"Confidence: {r.overall_confidence}")
print()
for c in r.candidates:
    print(f"  {c.match_score:5.1f}  {c.template_name} ({c.category})")

---

**MolBuilder** is an open-source, pure-Python molecular engineering toolkit.
No RDKit, no OpenBabel, no C++ dependencies.

- PyPI: `pip install molbuilder`
- GitHub: [github.com/Taylor-C-Powell/Molecule_Builder](https://github.com/Taylor-C-Powell/Molecule_Builder)
- API: [molbuilder-api.up.railway.app](https://molbuilder-api.up.railway.app)